## CS521 HW1

### Problem 1
#### 1.1
Here, the code snippet is related to calculate the pertubation. Since this is a targeted attack, we need to minimize the loss of the target class t. So here is my answer:

In [ ]:
adv_x = x - eps * x.grad.sign() # minimize the loss of the target class t

#### 1.2
When I let t=1, the result is 2 with an assertion error.
The first thing that I think about is to change the pertubation. I added the epsReal by 0.5, and it was classed to 2. When I added the epsReal by 1, it was classed to 0. So I adjusted epsReal to 0.5+0.7, and it was classed to 1 finally with tensor([[0.0331, 0.0381, 0.0349]], grad_fn=<MmBackward0>). Here, to measure the norm of the difference, I calculated the L2 distance: 3.7947328090667725. And for the targeted class 0 mentioned in 1.1, the L2 distance is 1.5811386108398438.
If I made epsReal = 0.5 + 0.64, the L2 distance would be 3.6049959659576416. This is the smallest one that I found. In this case, I believe FGSM attack is ineffective for the given network.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>


### Problem 2
#### 2.1
**Report**

pretr_Linf.pth
- Standard accuracy 0.85938
- Robust accuracy 0.51210 on pgd_linf attack with eps = 0.03137254901960784
- Robust accuracy 0.48830 on pgd_l2 attack with eps = 0.75

pretr_L2.pth:
- Standard accuracy 0.90625
- Robust accuracy 0.30850 on pgd_linf attack with eps = 0.03137254901960784
- Robust accuracy 0.54380 on pgd_l2 attack with eps = 0.75

pretr_RAMP.pth:
- Standard accuracy 0.84375
- Robust accuracy 0.49760 on pgd_linf attack with eps = 0.03137254901960784
- Robust accuracy 0.59650 on pgd_l2 attack with eps = 0.75

**Observations**
1. Model with weight pretr_L2 is the more accurate one with standard/clean accuracy 0.90625.
2. Model with weight pretr_Linf is the most robust one for pgd_linf attack with robust accuracy 0.51210.
3. Model with weight pretr_RAMP is the most robust one for pgd_l2 attack with robust accuracy 0.59650.

Actually I'm not quite sure about the k, I set it to 10 following the hint. I've attached the code snippets below.

In [ ]:
# 1. pgd_linf
# 1.1 implementation in function pgd_linf_untargeted
# TODO: Calculate the loss
loss = ce_loss(output, labels)
loss.backward()
# TODO: compute the adv_x
# find delta, clamp with eps
delta = eps_step * adv_x.grad.sign() # this is similar to fgsm
adv_x = adv_x + delta # gradient ascent to maximize the loss, this is for untargeted attack, so we need to increase the loss
new_delta = torch.clamp(adv_x - x, min=-eps, max=eps) # ensure the perturbation is within the eps range
adv_x = torch.clamp(x + new_delta, min=0, max=1).detach() # ensure the adv_x image is within the valid range [0, 1]

# 1.2 implementation in function test_model_on_single_attack
x_adv = pgd_linf_untargeted(model, x_batch, y_batch, k=10, eps=eps, eps_step=eps/4) # model is the preloaded model, x_batch is the input image batch, y_batch is the true label batch, k is set to 10 (actually I'm not sure why 10, maybe just a common choice. I just follow the hint), eps is 0.1, eps_step is eps/4

# Then I just run this
test_model_on_single_attack(model, attack='pgd_linf')

# 2. pgd_l2 implementation in function pgd_l2_untargeted
# 2.1 implementation in function pgd_linf_untargeted
# TODO: Calculate the loss
loss = ce_loss(output, labels)
grad = torch.autograd.grad(loss, adv_x, retain_graph=False,create_graph=False)[0] # get the gradient of the loss w.r.t. adv_x, this is different with pgd_linf
grad_norms = (torch.norm(grad.view(batch_size, -1), p=2,dim=1) + 1e-10)   # get the l2 norm
grad = grad / grad_norms.view(batch_size, 1, 1, 1)
# 2.2 compute the adv_x
delta = eps_step * grad # this is similar to fgsm, but the grad is normalized to have l2 norm of 1
adv_x = adv_x + delta # # this is for untargeted attack, so we need to increase the loss
new_delta = adv_x - x
new_delta_norms = torch.norm(new_delta.view(batch_size, -1), p=2, dim=1)
factor = eps / new_delta_norms # when the norm is greater than eps, then factor is less than 1, else factor is greater than 1
factor = torch.min(factor, torch.ones_like(new_delta_norms)) # if the new_delta_norms is greater than eps, then factor is 1, else factor is eps/norm. This make sure factor is at most 1.
new_delta = new_delta * factor.view(batch_size, 1, 1, 1) # ensure the perturbation is within the eps range
adv_x = torch.clamp(x + new_delta, min=0, max=1).detach() # ensure the adv_x image is within the valid range [0, 1]


x_adv = pgd_l2_untargeted(model, x_batch, y_batch, k=10, eps=eps, eps_step=eps/4)

In [ ]:
# Here is the code for standard evaluation

def check_clean(model, test_loader, device):
    x_batch, y_batch = next(iter(test_loader))
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    acc_raw = model(x_batch).argmax(1).eq(y_batch).float().mean().item()

    print(f"Standard accuracy %.5lf" % (acc_raw))

In [ ]:
# Here is my evaluation code

def eval_model():
    # intialize the model
    model = PreActResNet18(10, cuda=True, activation='softplus1').to(device)
    model.eval()
    model.load_state_dict(torch.load('models/pretr_Linf.pth'))
    model.eval()
    # Evaluate on clean examples
    check_clean(model, test_loader, device)
    # Evaluate on Linf attack with model 1 with eps = 8/255
    test_model_on_single_attack(model, attack='pgd_linf', eps=8/255)
    # Evaluate on Linf attack with model 1 with eps = 0.75
    test_model_on_single_attack(model, attack='pgd_l2', eps=0.75)

    model.load_state_dict(torch.load('models/pretr_L2.pth'))
    model.eval()
    # Evaluate on clean examples
    check_clean(model, test_loader, device)
    # Evaluate on Linf attack with model 2 with eps = 8/255
    test_model_on_single_attack(model, attack='pgd_linf', eps=8/255)
    # Evaluate on Linf attack with model 2 with eps = 0.75
    test_model_on_single_attack(model, attack='pgd_l2', eps=0.75)

    model.load_state_dict(torch.load('models/pretr_RAMP.pth'))
    model.eval()
    # Evaluate on clean examples
    check_clean(model, test_loader, device)
    # Evaluate on Linf attack with model 3 with eps = 8/255
    test_model_on_single_attack(model, attack='pgd_linf', eps=8/255)
    # Evaluate on Linf attack with model 3 with eps = 0.75
    test_model_on_single_attack(model, attack='pgd_l2', eps=0.75)

<br><br><br><br><br><br><br><br><br><br><br><br>

#### 2.2

 **Report**

 pretr_Linf.pth: Robust accuracy 0.51000 on multi attacks

 pretr_L2.pth: Robust accuracy 0.30850 on multi attacks

 pretr_RAMP.pth: Robust accuracy 0.49760 on multi attacks

 **Observation**
 1. Model with weight pretr_Linf is the most robust one against multiple attacks, pretr_RAMP is the second, and pretr_L2 is the last.

In [ ]:
def test_model_on_multi_attacks(model, eps_linf=8./255., eps_l2=0.75):
    model.eval()
    tot_test, tot_acc = 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        # TODO: get x_adv_linf and x_adv_l2 untargeted pgd linf and l2 with eps, and eps_step=eps/4
        x_adv_linf = pgd_linf_untargeted(model, x_batch, y_batch, k=10, eps=eps_linf, eps_step=eps_linf/4)
        x_adv_l2 = pgd_l2_untargeted(model, x_batch, y_batch, k=10, eps=eps_l2, eps_step=eps_l2/4)

        ## calculate union accuracy: correct only if both attacks are correct
        out = model(x_adv_linf)
        pred_linf = torch.max(out, dim=1)[1]
        out = model(x_adv_l2)
        pred_l2 = torch.max(out, dim=1)[1]

        # TODO: get the testing accuracy with multi-norm robustness and update tot_test and tot_acc
        tot_acc += ((pred_linf == y_batch) & (pred_l2 == y_batch)).sum().item()
        tot_test += y_batch.size(0)

    print('Robust accuracy %.5lf' % (tot_acc/tot_test), f'on multi attacks')

def eval_model_multi():
    # intialize the model
    model = PreActResNet18(10, cuda=True, activation='softplus1').to(device)
    model.eval()
    # Evaluate on L2 attack with different models with eps = 0.5
    model.load_state_dict(torch.load('models/pretr_Linf.pth'))
    model.eval()
    # Evaluate on multi attacks with model 1
    test_model_on_multi_attacks(model, eps_linf=8/255, eps_l2=0.5)

    model.load_state_dict(torch.load('models/pretr_L2.pth'))
    model.eval()
    # Evaluate on multi attacks with model 2
    test_model_on_multi_attacks(model, eps_linf=8/255, eps_l2=0.5)

    model.load_state_dict(torch.load('models/pretr_RAMP.pth'))
    model.eval()
    # Evaluate on multi attacks with model 3
    test_model_on_multi_attacks(model, eps_linf=8/255, eps_l2=0.5)

<br><br><br><br><br>
### Problem 3

#### Summary
The paper is mainly about building a benchmark for evaluating 19 types of unforseen adversaries.

#### Strengths
1. The topic is interesting. Unforseen robustness means the robustness for models that are not trained on specific type of attacks. For automotives, there can easily be adversary attacks (e.g., stop signs), and it is not possible to collect all of the attacks and train a model to against all of them. The research question is valuable and important.
2. The paper proposes a new metric called UA2 to measure the unforeseen robustness. The paper does provide more evaluation against more attacks, which makes UA2 more useful to assess the performance under more adversarial settings.
3. The paper explores methods to improve unforseen robustness, and discusses how training strategies affect unforeseen robustness. These provide insights for future works.
#### Weaknesses
1. The paper evaluates on 19 non-Lp attacks, but it is unclear how they are systematically collected, and whether they are representative of real-world scenarios. In subsection 4.2, for the 8 core attacks, although most are new, but it's not clear how they are likely to occur in real world settings. This is important because some models may perform well on some specific scenarios, there may be attacks other than the given ones, how to measure the completeness of the attacks?
2. For the evaluation, in subsection 5.1, the paper mentions that UA2 after optimisation is similar to worst case robustness, and similar to L_p robust accuracy, why it is a new metric? Furthermore, I'm curious about what optimisation makes UA2 similar to worst case robustness? It seems the authors didn't provide any explaination.Besides, the paper mentions that "We conclude that UA2 is distinct from L_p robustness, and present UA2 as an improved progress measure when working towards real-world worst-case robustness." However, is it really an improved progress measure, or just behaves different for specific scenarios?
3. The paper doesn't seem self-contained. I find it relies heavily on the appendix. For example, in section 5, the paper simply discussed the differences with other works like ImageNet-C and other works, but the quantitative results are all in the appendix. But the differences with previous works are very important, right? Besides, the paper has done a lot of measurement works, but

#### Extensions
I'm curious about how the proposed approach and metric could be applied to real-world scenarios, such as automotives, robotics, and other safety-related domains. These domains involves richer contexts and complex sources of unforeseen adversaries, which may have overlap with the ones discussed in the paper. Exploring UA2 in such scenarios help validate whether it indeed is a useful metric, or whether it needs further extend to domain-specific constraints. This would enlarge the real-world impact of this work.